# Task 1. Extracting text features. Preprocessing

## Imports

In [1]:
import re

import pandas as pd

from natasha import Doc
from natasha import MorphVocab
from natasha import Segmenter
from natasha import NewsEmbedding
from natasha import NewsMorphTagger

import nltk
nltk.data.path = ["../nltk_data"]
nltk.download("stopwords", download_dir="../nltk_data")
from nltk.corpus import stopwords

import tqdm

[nltk_data] Downloading package stopwords to ../nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stopwords = stopwords.words("russian")

## Data preprocessing

In [3]:
df = pd.read_csv("../../data/petitions.csv")

In [4]:
df.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [5]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [6]:
df.drop("id", axis=1, inplace=True)

In [7]:
df["reason_category"].unique()

array(['Благоустройство', 'Содержание МКД',
       'Незаконная информационная и (или) рекламная конструкция', 'Фасад',
       'Водоснабжение', 'Нарушение правил пользования общим имуществом',
       'Повреждения или неисправность элементов уличной инфраструктуры',
       'Кровля', 'Состояние рекламных или информационных конструкций',
       'Нарушение порядка пользования общим имуществом', 'Подвалы',
       'Водоотведение', 'Санитарное состояние', 'Центральное отопление',
       'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)'],
      dtype=object)

In [8]:
to_replace = {
    "Фасад": "Содержание МКД",
    "Водоснабжение": "Содержание МКД",
    "Кровля": "Содержание МКД",
    "Подвалы": "Содержание МКД",
    "Водоотведение": "Содержание МКД",
    "Санитарное состояние": "Содержание МКД",
    "Центральное отопление": "Содержание МКД",

    "Повреждения или неисправность элементов уличной инфраструктуры": "Благоустройство",
    "Состояние рекламных или информационных конструкций": "Благоустройство",

    "Незаконная информационная и (или) рекламная конструкция": "Нарушение",
    "Нарушение правил пользования общим имуществом": "Нарушение",
    "Нарушение порядка пользования общим имуществом": "Нарушение",
    "Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)": "Нарушение",
}

df.replace(to_replace, inplace=True)

In [9]:
df["public_petition_text"] = df["public_petition_text"].apply(str.lower)
df["reason_category"] = df["reason_category"].apply(str.lower)

In [10]:
df["public_petition_text"] = df["public_petition_text"].apply(lambda s: re.sub(r"[^\s^\w]+", " ", s))

In [11]:
df["public_petition_text"] = df["public_petition_text"].apply(lambda s: s.replace("\n", ""))

In [12]:
df.head()

,public_petition_text,reason_category
0,снег на дороге,благоустройство
1,очистить кабельный киоск от рекламы,благоустройство
2,просим убрать все деревья и кустарники которы...,благоустройство
3,неудовлетворительное состояние парадной надп...,содержание мкд
4,граффити,благоустройство


## Text preprocessing

In [13]:
morph_vocab = MorphVocab()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

#### Lemmatization

In [14]:
def lemmatize(s: str) -> str:
    lemmas = []

    doc = Doc(s)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemmas.append(token.lemma)

    return " ".join(lemmas)

In [15]:
for i, s in enumerate(tqdm.tqdm(df["public_petition_text"], bar_format="[+] Lemmatizing: |{bar:50}|")):
    df["public_petition_text"][i] = lemmatize(s)

[+] Lemmatizing: |██████████████████████████████████████████████████|


#### Removing stop words

In [16]:
def remove_stop_words(s: str) -> str:
    return " ".join([word for word in s.split() if word not in stopwords])

In [17]:
for i, s in enumerate(tqdm.tqdm(df["public_petition_text"], bar_format="[+] Removing stop words: |{bar:50}|")):
    df["public_petition_text"][i] = remove_stop_words(s)

[+] Removing stop words: |██████████████████████████████████████████████████|


#### Saving preprocessed dataset

In [18]:
df.head()

,public_petition_text,reason_category
0,снег дорога,благоустройство
1,очистить кабельный киоск реклама,благоустройство
2,просить убрать весь дерево кустарник который в...,благоустройство
3,неудовлетворительный состояние парадный надпис...,содержание мкд
4,граффити,благоустройство


In [19]:
df.to_csv("../../data/preprocessed_petitions.csv")